## 0726 NEW DB(시각화전용)
    * 목적 : 기존 0630 DB로 예측은 가능하지만 시각화에 있어 몇몇 FEATURE가 부족해서 새로운 DB 구축 필요
    * 새로 추가된 FEATURE
        - nnames : 펀딩을 한 회사 이름
        - nfund : 펀딩 목표 금액
        - nstart : 펀딩 시작 날짜
        - nend : 펀딩 끝나는 날짜
        - nnumofpics : 사용 사진 갯수
        - ncommunity : 커뮤니티 댓글 갯수
    * 추가한 이유 : 시각화 특화를 위해
    * 코드 전반적 설명
        - 무한 스크롤을 빼고

In [1]:
import requests
from bs4 import BeautifulSoup as bsp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

import json
import time
from datetime import datetime, timedelta
from selenium import webdriver

In [2]:
# PYTHON으로 SQLITE3의 DB에 데이터 담기
dbname='20210630_wadizDB_최종.db'
def insData(d, tblname='20210729_wadiztable'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select * from '+tblname+' where ncategory="'+d+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            return ckkey

In [3]:
dbname1='20210726_newDB.db'
def insData1(d, tblname='wadiznew'):
    with sqlite3.connect(dbname1) as conn:
        cur=conn.cursor()
        sql='select nkey from '+tblname+' where nkey="'+d[-1]+'"'
        ckkey=cur.execute(sql).fetchall()
        if ckkey:
            print('-',end='')
        
        else:
            insql='insert into '+tblname+' (ncategory,nname,ntitle,ncontent,nsupport,namount,npercent,nfund,nstart,nend,nnumofpics,ncommunity,nkey) values (?,?,?,?,?,?,?,?,?,?,?,?,?)'
            cur.execute(insql,d)
            print('.',end='')

In [4]:
# df 뽑아내는 함수
def category(i):
    cate=['테크·가전', '패션·잡화', '뷰티', '푸드', '홈리빙', '여행·레저', '스포츠·모빌리티', '공연·컬쳐', 
          '베이비·키즈', '반려동물', '게임·취미', '클래스', '출판', '디자인소품', '소셜·캠페인', '기부·후원', '모임']
    data=insData(cate[i])
    df=pd.DataFrame(data)
    names=['idx','category','title','content','supporter','amount','percent','key']
    df.columns=names
    return df

# 스포츠모빌리티 선택(6)
df=category(0)
print('data는 ', len(df), '개')

data는  2260 개


In [5]:
for i in range(len(df)):
    url = df['key'][i]

    reqheader = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'} 
    src=requests.get(url,headers=reqheader)
    cont=src.content.strip() 
    html=bsp(cont,'html.parser')
    
    try:
        # 카테고리
        category=html.select('p.title-info em')
        ncategory=category[0].text
        
        # 회사명
        nname=html.find(id="makerNickName")['content']
        
        # 제목
        title=html.select('h2.title a')
        ntitle=title[0].text
        
        # 컨텐츠
        contents=html.select('.inner-contents p')
        summary=html.select('.campaign-summary')
        cont=[]

        for content in contents:
            cont.append(content.text)
        ncontent=summary[0].text+' '.join(cont)
        
        # 서포터 수
        support=html.select('p.total-supporter strong')
        nsupport=support[0].text
        
        # 금액 
        amount=html.select('p.total-amount strong')
        namount=amount[0].text

        percent=html.select('p.achievement-rate strong')
        npercent=percent[0].text

        object=html.find_all(style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;')[0].text
        funding=object.split('\n')

        nfund=funding[1].split()[2]

        nstart=funding[2].split()[2][-21:-11]
        nstart=nstart.replace('.','-')

        nend=funding[2].split()[2][-10:]
        nend=nend.replace('.','-')

        img_url = []
        imgs = html.find_all('img')
        for img in imgs:
            iurl = img.get('data-src')

            if iurl != None:
                img_url.append(iurl)
            else:
                iurl = img.get('src')
                img_url.append(iurl)
        nnumofpics = len(img_url)

        comm = html.find_all('span','count-total comment-total')
        if len(comm) == 0:
            ncommunity = 0
        else:
            ncommunity = comm[0].text

        nkey= url

        d=(ncategory,nname,ntitle,ncontent,nsupport,namount,npercent,nfund,nstart,nend,nnumofpics,ncommunity,nkey)
        insData1(d)
    except IndexError:
        print('No data')

......................................................................................................

KeyboardInterrupt: 